In [9]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X - X.mean()

In [10]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))


tensor([-2., -1.,  0.,  1.,  2.])

In [11]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [12]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(5.5879e-09, grad_fn=<MeanBackward0>)

In [13]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [14]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 2.2579,  0.2359,  0.0817],
        [ 1.0586,  0.2782,  0.2088],
        [-1.5001,  1.0587, -0.7873],
        [-1.2118, -0.0624,  0.6446],
        [ 1.3206,  0.1760,  1.5502]], requires_grad=True)

In [15]:
linear(torch.rand(2, 5))

tensor([[1.3516, 0.7494, 0.0000],
        [0.8219, 1.3910, 0.0000]])

In [17]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[15.0010],
        [17.5599]])

In [18]:
class HalfFFT(nn.Module):
    def __init__(self):
        super(HalfFFT, self).__init__()

    def forward(self, X):
        """
        Compute FFT and return half of it
        :param X: size = B*L
        :return: size = B*round(L/2)
        """
        half_len = round(X.shape[1]/2)
        X_f = torch.fft.fft(X)
        return X_f[:, :half_len]


myNet2 = HalfFFT()
print(myNet2(torch.rand(2, 3)))

tensor([[ 1.7783+0.0000j,  0.6029+0.5496j],
        [ 0.6178+0.0000j, -0.0487-0.1845j]])


In [19]:
class DimensionReduction(nn.Module):
    def __init__(self, i, j, k):
        super(DimensionReduction, self).__init__()
        self.net = nn.Conv2d(in_channels=1, out_channels=k, kernel_size=(i, j))

    def forward(self, X, Y):
        # 先用X和Y做矩阵乘法构成i*j矩阵，
        # 再用卷积层快捷地实现计算功能
        matrix = torch.bmm(x, torch.transpose(y, 1, 2))
        matrix = matrix.unsqueeze(1)  # B*1*i*j
        return self.net(matrix)  # B*5*i*j


myNet1 = DimensionReduction(2, 3, 5)
x = torch.ones(1, 2, 1)  # B*i*1
y = torch.rand(1, 3, 1)  # B*j*1
print(myNet1(x, y))

tensor([[[[-0.2451]],

         [[-0.3820]],

         [[ 0.3166]],

         [[-0.1738]],

         [[ 0.4742]]]], grad_fn=<SlowConv2DBackward0>)
